In [1]:
!pip install nltk
!pip install newspaper3k

In [2]:
# Import library
from newspaper import Article
import random
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
nltk.download('punkt', quiet = True)

True

## Proses 1 (Membuat korpus dengan mendownload artikel dari link)
Link : https://warstek.com/kertas-stone/

In [4]:
url_1 = 'https://gamezero.id/5-fakta-ilmu-sains-dari-dr-stone/'
url_2 = 'https://warstek.com/kertas-stone/'
url_3 = 'https://www.kompasiana.com/daniram/60f624961525102f527388a3/dr-stone-chapter-205-sai-manusia-dengan-kemapuan-otak-setara-satu-juta-orang'

In [5]:
article = Article(url_1)
article.download()
article.parse()
article.nlp()
corpus_1 = article.text

article = Article(url_2)
article.download()
article.parse()
article.nlp()
corpus_2 = article.text

article = Article(url_3)
article.download()
article.parse()
article.nlp()
corpus_3 = article.text

corpus = corpus_1, corpus_2, corpus_3
print(corpus)

('Apakah kalian ada mempelajari ilmu sains disekolah?\n\nApakah kalian tahu perbedaan antara asam amino dan enzim? Atau apakah kalian pernah mencoba berjuang membakar kayu menggunakan kedua batu?\n\nSerial anime ini menceritakan kisah ilmuwan Senku Ishigami yang berubah menjadi batu ketika seluruh umat manusia membatu setelah peristiwa misterius.\n\nSekitar 3700 tahun kemudian, ia bangun dan memutuskan untuk membangun kembali peradaban menggunakan sains dengan bantuan segelintir karakter lain yang berhasil dihidupkannya kembali. Jadi bagaimana menurut kalian? apakah penggunaan ilmu sains di anime Dr Stone akan sama seperti Ilmu Sains yang seharusnya di terapkan di bumi? mari kita simak artikel berikut!\n\nSenku membuat sabun\n\nDalam episode kedua, Senku menghancurkan beberapa kerang untuk mendapatkan kalsium karbonat untuk membuat sabun. Ini adalah komoditas penting karena tidak ada desinfektan di planet ini dan sabun adalah hal terbaik dalam hal kebersihan dan menangkal penyakit.\n\n

## Proses 2 (Tokenization)

In [6]:
import re
# Tokenization
text = str(corpus)
text = re.sub(r'\\[\\n]', ' ', text)
text = re.sub(r'[-()\\/\'/"#/@;:<>{}`+=~$|]', '', text)
sentence_list = nltk.sent_tokenize(text) # A list of sentences

# Cetak list sentences
print(sentence_list)

['Apakah kalian ada mempelajari ilmu sains disekolah?', 'Apakah kalian tahu perbedaan antara asam amino dan enzim?', 'Atau apakah kalian pernah mencoba berjuang membakar kayu menggunakan kedua batu?', 'Serial anime ini menceritakan kisah ilmuwan Senku Ishigami yang berubah menjadi batu ketika seluruh umat manusia membatu setelah peristiwa misterius.', 'Sekitar 3700 tahun kemudian, ia bangun dan memutuskan untuk membangun kembali peradaban menggunakan sains dengan bantuan segelintir karakter lain yang berhasil dihidupkannya kembali.', 'Jadi bagaimana menurut kalian?', 'apakah penggunaan ilmu sains di anime Dr Stone akan sama seperti Ilmu Sains yang seharusnya di terapkan di bumi?', 'mari kita simak artikel berikut!', 'Senku membuat sabun  Dalam episode kedua, Senku menghancurkan beberapa kerang untuk mendapatkan kalsium karbonat untuk membuat sabun.', 'Ini adalah komoditas penting karena tidak ada desinfektan di planet ini dan sabun adalah hal terbaik dalam hal kebersihan dan menangkal 

In [7]:
# Membuat fungsi untuk mengembalikan respon sapaan ke pengguna
def greeting_response(text):
  text = text.lower()

  # Respon sapaan dari bot
  bot_greeting = ['halo', 'hai', 'iya, ada apa?', 'ada yang bisa saya bantu?', 'selamat datang']

  # Sapaan pengguna
  user_greeting = ['permisi', 'halo', 'hai', 'salam', 'wassup', 'konichiha', 'konichiwa']

  for word in text.split():
    if word in user_greeting:
      return random.choice(bot_greeting)

  # Respon sapaan secara acak dibuat huruf kecil semua
  def gratitude_response():
    text = text.lower()

In [8]:
# Mengurutkan Variabel
def index_sort(list_var):
  length = len(list_var)
  list_index = list(range(0, length))

  x = list_var
  for i in range(length):
    for j in range(length):
      if x[list_index[i]] > x[list_index[j]]:

        # Menukar
        temp = list_index[i]
        list_index[i] = list_index[j]
        list_index[j] = temp

  return list_index

In [9]:
# Membuat respon bots
def bot_response(user_input):
  user_input = user_input.lower() # Merubah input dari user menjadi huruf kecil
  sentence_list.append(user_input) # Memasukan input user ke dalam korpus yang kita buat (Korpus terletak pada variabel sentence_list)
  bot_response = '' # Inisialisasi variabel dari bot response

  cm = CountVectorizer().fit_transform(sentence_list) # Melakukan ekstraksi fitur pada korpus menggunakan CountVectorizer (Bag of Word)/Mengubah santence_list menjadi vektor

  similarity_score = cosine_similarity(cm[-1], cm) # Menghitung cosine similarity antara hasil fitur ekstraksi input user (cm[-1]) : Mengambil elmen terahir, dengan semua data dalam korpus
  similarity_score_list = similarity_score.flatten() #Mengubah hasil perhitungan cosine similarity menjadi bentuk list 1D
  index = index_sort(similarity_score_list) # Mengurutkan index pada similarity score berdasarkan elmen yang terbesar, kemudian hasilnya disimpan pada list bernama index
  index = index[1:] # Mengambil nilai terbesar ke 2 karena jika yang pertama maka akan sama seperti input.

  response_flag = 0
  for i in range(len(index)):
    if similarity_score_list[index[i]] > 0.0: # Jika similarity_score_list pada index lebih dari 0 maka kemiripannya lebih besar
      bot_response = bot_response+' '+sentence_list[index[i]] # Respon dari bot
      response_flag = 1

    if response_flag == 0:
      bot_response = bot_response+" "+"Maaf hamba belum mengerti" # Jika response_flag = 0 (tidak ada yang cocok) maka akan memberikan respon minta maaf

    sentence_list.remove(user_input) # Menghapus user_input yg sudah dimasikan ke dalam santence_list (Korpus)

    return bot_response

## Proses 4 (end Process, Membuat chat box)

In [10]:
#Start Chat
print("Weebs Bot: Mau nanya apa kamu tentang kepintaran Senku?")

exit_list = ['exit','bye','keluar','quit', 'sampai jumpa', 'sayounara']

while(True):
    user_input = input('User : ') #mengambil input pada user
    if user_input.lower() in exit_list: #kalau input dari user ada didalam exit_list maka program akan berhenti
        print('Weebs Bot: Bye Bye Sampai Jumpa Lagi')
        print('Weebs Bot: Selamat anda menjadi WIBU!!!')
        break
    else:
        if greeting_response(user_input)!= None: #Mengecek apakah input dari user masuk kedalam greeting atau bukan
            print('Weebs Bot: '+ greeting_response(user_input)) #Jika masuk kedalam greeting maka bot akan melakukan response terhadap greeting tersebut
        else:
            print('Weebs Bot: '+ bot_response(user_input)) #Jika bukan, maka bot akan memanggil fungsi bot_response untuk menentukan respon apa yg akan ia keluarkan sesuai dengan input dari user

Weebs Bot: Mau nanya apa kamu tentang kepintaran Senku?
User : konichiha
Weebs Bot: hai
User : besi tahan panas?
Weebs Bot:  Membuat Telepon  Membuat perangkat komunikasi modern di Zaman Batu bukanlah hal yang mudah, tetapi Senku menemukan tungsten, logam paling tahan panas di alam semesta.
User : keluar
Weebs Bot: Bye Bye Sampai Jumpa Lagi
Weebs Bot: Selamat anda menjadi WIBU!!!
